In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

%matplotlib inline

In [7]:
import torch

print(torch.cuda.is_available())     # Returns a bool indicating if CUDA is currently available.
print(torch.cuda.current_device())   # Returns the index of a currently selected device.
print(torch.cuda.device(0))          # Context-manager that changes the selected device.
print(torch.cuda.device_count())     # Returns the number of GPUs available.
print(torch.cuda.get_device_name(0)) # Gets the name of a device.

True
0
1
NVIDIA GeForce RTX 3090


In [4]:
df=pd.read_csv('datasets/data.csv')
df[['q_mu', 'q_nu1', 'q_nu2', 'e_nu_1', 'e_nu_2']]

,q_mu,q_nu1,q_nu2,e_nu_1,e_nu_2
0,0.550377,0.164911,0.008094,0.01904,0.17794
1,0.564295,0.167629,0.006639,0.01672,0.20868
2,0.564179,0.164110,0.007052,0.01337,0.20442
3,0.573555,0.167174,0.006663,0.01637,0.18453
4,0.569296,0.169658,0.006823,0.01783,0.11478
...,...,...,...,...,...
184845,0.566484,0.172819,0.008164,0.02085,0.25589
184846,0.572031,0.170474,0.007907,0.01592,0.23668
184847,0.569204,0.173147,0.007366,0.01590,0.24216
184848,0.570114,0.168854,0.007378,0.01384,0.24447


In [ ]:
data=np.log(df['e_mu_current'])
data_std=np.std(data)
data_mean=np.mean(data)

labels=[]

for point in data:
    if point >=2*data_std+data_mean:
        labels.append(1)
    else:
        labels.append(0)

labels=np.array(labels)

In [ ]:
def BoxDataset(Dataset):
    def __init__(self, df_path):
        self.data=pd.read_csv(df_path)
        self.x = self.data[['q_mu', 'q_nu1', 'q_nu2', 'e_nu_1', 'e_nu_2']]
        self.boxes=[]
        
        log_data=np.log(self.data['e_mu_current'])
        data_std=np.std(log_data)
        data_mean=np.mean(log_data)

#         labels=[]
#         for point in data:
#             if point >=2*data_std+data_mean:
#                 labels.append(1)
#             else:
#                 labels.append(0)

#         self.labels=np.array(labels)
    def __len__(self):
        return len(self.boxes)

    def __getitem__(self, idx):
        return self.boxes[idx], self.labels[idx]